In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.linalg as linalg
import os
import time
from ParticleFilter import sample_normal_model, single_step_particle_filter, normal_model_pdf, single_step_particle_filter_measurement_window
from utils.plotting import draw_ellipse, images_to_gif
from utils.filesTools import get_exp_folder

In [ ]:
## setup params
NUM_OF_BEACONS = 4
NUM_OF_AGENTS = 1
STATE_SIZE_2D = 2
SINGLE_RANGE_MEASUREMENT_SIZE = 1
RANGE_MEASUREMENT_SIZE = NUM_OF_BEACONS * SINGLE_RANGE_MEASUREMENT_SIZE * NUM_OF_AGENTS
TOTAL_STATE_SIZE = NUM_OF_AGENTS * STATE_SIZE_2D + NUM_OF_BEACONS * STATE_SIZE_2D
sigma_transition_agent = 0.5
sigma_transition_beacon = 0.3
sigma_measurement = 0.1
stepsize = 2
n_steps = 150
n_particles = 100
# model definition
cov_measurement = np.diag([sigma_measurement**2 for i in range(RANGE_MEASUREMENT_SIZE)])
cov_transition_agent = [sigma_transition_agent**2 for i in range(NUM_OF_AGENTS*STATE_SIZE_2D)]
cov_transition_agent = [0 for i in range(NUM_OF_AGENTS*STATE_SIZE_2D)]
cov_transition_beacon = [sigma_transition_beacon**2 for i in range(NUM_OF_BEACONS*STATE_SIZE_2D)]
# cov_transition_beacon = [0 for i in range(NUM_OF_BEACONS*STATE_SIZE_2D)]
# cov_transition_beacon[0:6] = [sigma_transition_beacon**2]*6
cov_transition = np.diag(cov_transition_agent + cov_transition_beacon)


In [ ]:
# state manger
get_agent_index = lambda i: slice(i*STATE_SIZE_2D, i*STATE_SIZE_2D + STATE_SIZE_2D)
get_beacon_index = lambda i: slice(STATE_SIZE_2D*NUM_OF_AGENTS + i*STATE_SIZE_2D, STATE_SIZE_2D*NUM_OF_AGENTS + i*STATE_SIZE_2D + STATE_SIZE_2D)
get_agent_position = lambda x, i: x[get_agent_index(i)]
get_beacon_postion = lambda x, j: x[get_beacon_index(j)]
def state_to_agent_and_beacons_pos(x):
    agents_pos = np.zeros((NUM_OF_AGENTS, STATE_SIZE_2D))
    beacons_pos = np.zeros((NUM_OF_BEACONS, STATE_SIZE_2D))
    for i in range(NUM_OF_AGENTS):
        agents_pos[i] = get_agent_position(x, i)
    for j in range(NUM_OF_BEACONS):
        beacons_pos[j] = get_beacon_postion(x, j)
    return agents_pos, beacons_pos
def agent_and_beacons_pos_to_state(agents_pos, beacons_pos):
    x = np.zeros(TOTAL_STATE_SIZE)
    for i in range(NUM_OF_AGENTS):
        x[get_agent_index(i)] = agents_pos[i]
    for j in range(NUM_OF_BEACONS):
        x[get_beacon_index(j)] = beacons_pos[j]
    return x
## model definition
'''
Function to propagate the state based on the control input and a normal model
'''
propagate_state_function = lambda x, u: x + sample_normal_model(u, cov_transition)


def calculate_true_range_meas(x):
    """
    Function to calculate the true range measurements.
    It calculates the Euclidean distance between each agent and each beacon.
    """
    z = np.zeros(RANGE_MEASUREMENT_SIZE)
    for i in range(NUM_OF_AGENTS):
        current_agent_position = x[get_agent_index(i)]
        for j in range(NUM_OF_BEACONS):
            current_beacon_position = x[get_beacon_index(j)]
            z[i*NUM_OF_BEACONS + j] = np.linalg.norm(current_agent_position - current_beacon_position)
    return z

def measurements_model(x, cov = cov_measurement):
    '''
    Function to generate the range measurements model.
    It adds a normally distributed noise to the true range measurements.
    '''
    return calculate_true_range_meas(x) + sample_normal_model(np.zeros(RANGE_MEASUREMENT_SIZE), cov)

'''
function to calculates the likelihood of the measurements given the state.
It uses a normal probability density function with the true range measurements
as the mean and the measurement covariance as the covariance
'''
measurements_likelihood = lambda z, x: normal_model_pdf(z, calculate_true_range_meas(x), cov_measurement)
## test models and conversion functions
if 0:
    x = np.array([0, 0, 1, 0, 2, 0, 3, 0, 4, 0])*100
    u = np.zeros(TOTAL_STATE_SIZE)
    print(measurements_model(x))
    print(propagate_state_function(x, u))
## test conversion functions
if 0:
    x = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])
    agents_pos, beacons_pos = state_to_agent_and_beacons_pos(x)
    print(agents_pos, beacons_pos)
    print(agent_and_beacons_pos_to_state(agents_pos, beacons_pos))
# pf Utils functions
def draw_frame(x,particles,frames_folder,i,ellipse = True):
    colors = ['r', 'g', 'b', 'y', 'm']
    fig, ax = plt.subplots()
    for j in range(NUM_OF_AGENTS):
        agent_gt_pos = x[get_agent_index(j),i]
        agent_particles = particles[:,get_agent_index(j)]
        agent_color = colors[j]
        ax.scatter(agent_gt_pos[0], agent_gt_pos[1], color = agent_color)
        if ellipse:
            draw_ellipse(ax, edgecolor = agent_color ,data = agent_particles)
        else:
            ax.scatter(agent_particles[:,0], agent_particles[:,1], c = agent_color, marker = 'x')
            
    for j in range(NUM_OF_BEACONS):
        beacon_gt_pos = x[get_beacon_index(j),i]
        beacon_particles = particles[:,get_beacon_index(j)]
        beacon_color = colors[j+NUM_OF_AGENTS]
        ax.scatter(beacon_gt_pos[0], beacon_gt_pos[1], color = beacon_color)
        if ellipse:
            draw_ellipse(ax, edgecolor= beacon_color ,data = beacon_particles)
        else:
            ax.scatter(beacon_particles[:,0], beacon_particles[:,1], c = beacon_color, marker = 'x')
    
    plt.xlim(-20, 20)
    plt.ylim(-20, 20)
    plt.title(f'frame {i}')
    plt.savefig(f'{frames_folder}/frame_{i}.png')
    plt.close()
    
def calculate_mean_and_cov(particles):
    mean = np.mean(particles, axis = 0)
    cov = np.cov(particles.T).flatten()
    return mean, cov

def save_ground_truth(x, z, exp_folder):
    pd.DataFrame(x).to_csv(f'{exp_folder}/ground_truth_state.csv')
    pd.DataFrame(z).to_csv(f'{exp_folder}/ground_truth_measurement.csv')

def save_current_particles(particles, exp_folder, i):
    if not os.path.exists(exp_folder):
        os.makedirs(exp_folder)
    if not os.path.exists(f'{exp_folder}/particles'):
        os.makedirs(f'{exp_folder}/particles')
    pd.DataFrame(particles).to_csv(f'{exp_folder}/particles/particles_{i}.csv')

def run_particle_filter_experiment(note, particles, n_steps, total_state_size, x, u, z, propagate_state_function, measurements_likelihood_function, resample_method='systematic', save_particles=False, ellipse=True):
    exp_folder, frames_folder = get_exp_folder(note = note)
    mean_log = np.zeros((n_steps, total_state_size))
    cov_log = np.zeros((n_steps, total_state_size**2))

    for i in range(n_steps):
        if 1:
            draw_frame(x, particles, frames_folder, i, ellipse = ellipse)
        if save_particles:
            save_current_particles(particles, exp_folder, i)
        
        particles = single_step_particle_filter(particles, u[:,i], z[:,i], propagate_state_function, measurements_likelihood_function, resample_method = resample_method)
        mean_log[i], cov_log[i] = calculate_mean_and_cov(particles)
        print(f"\r{i}/{n_steps}", end='', flush=True)
    print('\n')
    if 1:
        draw_frame(x, particles, frames_folder, i = n_steps - 1, ellipse = ellipse)
    if save_particles:
        save_current_particles(particles, exp_folder, i)
    images_to_gif(frames_folder, f'{exp_folder}/particle_filter.gif')
    pd.DataFrame(mean_log).to_csv(f'{exp_folder}/mean_log.csv', index = False, header = False)
    pd.DataFrame(cov_log).to_csv(f'{exp_folder}/cov_log.csv', index = False, header = False)
    save_ground_truth(x, z, exp_folder)

    return mean_log, cov_log, exp_folder, particles

In [ ]:
## generate ground truth
agent_position_0 = np.array([0,0])
becons_1_position_0 = np.array([10,10])
becons_2_position_0 = np.array([-10,10])
becons_3_position_0 = np.array([10,-10])
becons_4_position_0 = np.array([-10,-10])
x_0 = agent_and_beacons_pos_to_state([agent_position_0], [becons_1_position_0, becons_2_position_0, becons_3_position_0, becons_4_position_0])

#genrate u  commend vector
u = (np.random.rand(TOTAL_STATE_SIZE, n_steps) - 0.5) * stepsize
u = np.zeros((TOTAL_STATE_SIZE, n_steps))
u[get_agent_index(0),:] = (np.random.rand(STATE_SIZE_2D, n_steps) - 0.5) * stepsize
u[get_beacon_index(0),:] = (np.random.rand(STATE_SIZE_2D, n_steps) - 0.5) * stepsize/10
u[get_beacon_index(1),:] = (np.random.rand(STATE_SIZE_2D, n_steps) - 0.5) * stepsize/10
u[get_beacon_index(2),:] = (np.random.rand(STATE_SIZE_2D, n_steps) - 0.5) * stepsize/10
u[get_beacon_index(3),:] = (np.random.rand(STATE_SIZE_2D, n_steps) - 0.5) * stepsize/10
## create ground truth
x = np.zeros((TOTAL_STATE_SIZE, n_steps))
x[:,0] = x_0
z = np.zeros((RANGE_MEASUREMENT_SIZE, n_steps))
z[:,0] = measurements_model(x[:,0])
for i in range(1, n_steps):
    x[:,i] = propagate_state_function(x[:,i-1], u[:,i-1])
    z[:,i] = measurements_model(x[:,i])

## plot ground truth
if 0:
    fig, ax = plt.subplots()
    ax.plot(x[0,:], x[1,:], 'r')
    ax.plot(x[2,:], x[3,:], 'g')
    ax.plot(x[4,:], x[5,:], 'b')
    ax.plot(x[6,:], x[7,:], 'y')
    ax.plot(x[8,:], x[9,:], 'm')
    plt.show()


In [ ]:
## particle filter paerameters and initialization
NOTE = 'circle initialization - systematic resampling'
SAVE_EACH_FRAME = 1
n_particles = 1000
particles = (np.random.rand(n_particles, TOTAL_STATE_SIZE) - 0.5) * 15
particles[:,get_agent_index(0)] = np.zeros((n_particles, STATE_SIZE_2D))
r = np.linalg.norm(becons_1_position_0)
theta = np.linspace(0, 2*np.pi, n_particles)
beacons_pos_0 = np.array([r*np.cos(theta), r*np.sin(theta)]).T
particles[:,get_beacon_index(0)] = beacons_pos_0
particles[:,get_beacon_index(1)] = beacons_pos_0
particles[:,get_beacon_index(2)] = beacons_pos_0
particles[:,get_beacon_index(3)] = beacons_pos_0
systematic_output = run_particle_filter_experiment(note = NOTE,
                                                    n_steps= n_steps,
                                                    particles = particles,
                                                    total_state_size = TOTAL_STATE_SIZE,
                                                    x = x,
                                                    u = u,
                                                    z = z,
                                                    propagate_state_function=propagate_state_function,
                                                    measurements_likelihood_function = measurements_likelihood,
                                                    resample_method='systematic',
                                                    save_particles=SAVE_EACH_FRAME,
                                                    ellipse=True)

In [ ]:
#calc errors
if 0:
    errors = np.zeros((n_steps, TOTAL_STATE_SIZE))
    for i in range(n_steps):
        errors[i] = x[:,i] - mean_log[i]
    pd.DataFrame(errors).to_csv(f'{exp_folder}/errors.csv', index = False, header = False)

    #plot errors, split by aagent and beacons in different subplots for each agent and beacon
    if 1:
        fig, axs = plt.subplots(NUM_OF_AGENTS + NUM_OF_BEACONS, 1)
        for i in range(NUM_OF_AGENTS):
            axs[i].plot(np.linalg.norm(errors[:,get_agent_index(i)],axis = 1))
            axs[i].set_title(f'agent {i} errors')
        for j in range(NUM_OF_BEACONS):
            axs[NUM_OF_AGENTS + j].plot(np.linalg.norm(errors[:,get_beacon_index(j)],axis = 1))
            axs[NUM_OF_AGENTS + j].set_title(f'beacon {j} errors')
        plt.show()
        
    # plot the measermnt resedules
    meas_residual = np.zeros((n_steps, RANGE_MEASUREMENT_SIZE))
    for i in range(n_steps):
        meas_residual[i] = calculate_true_range_meas(mean_log[i]) - calculate_true_range_meas(x[:,i])
    if 1:
        fig, axs = plt.subplots(RANGE_MEASUREMENT_SIZE, 1)
        for i in range(RANGE_MEASUREMENT_SIZE):
            axs[i].plot(meas_residual[:,i])
            axs[i].set_title(f'measurement {i} residuals')
        plt.show()
    # put state error and meas error in the same subplots
    if 1:
        fig, axs = plt.subplots(NUM_OF_BEACONS, 1)
        for i in range(NUM_OF_BEACONS):
            axs[i].plot(np.linalg.norm(errors[:,get_beacon_index(i)],axis = 1))
            axs[i].plot(meas_residual[:,i])
            # axs[i].set_title(f'state {i} errors')
        plt.legend(['state error', 'meas error'])
        plt.show()